In [117]:
import requests
import pandas as pd

In [118]:
FOLDER_RAW = "./data/raw"

In [119]:
# Extracting and saving info about the Copom Minutes from the Brazilian Central Bank API.
url = "https://www.bcb.gov.br/api/servico/sitebcb/copom/atas?quantidade=1000"

response = requests.get(url)


In [120]:
df_minutes_date = pd.DataFrame(response.json()["conteudo"])
df_minutes_date["dataReferencia"] = pd.to_datetime(df_minutes_date["dataReferencia"], format="%Y-%m-%d").dt.date
df_minutes_date["dataPublicacao"] = pd.to_datetime(df_minutes_date["dataPublicacao"], format="%Y-%m-%d").dt.date
df_minutes_date.set_index("nroReuniao", inplace=True)

df_minutes_date = df_minutes_date.tail(len(df_minutes_date) - 1)
df_minutes_date = df_minutes_date.head(len(df_minutes_date) - 21)

In [121]:
df_info = pd.read_excel(f"{FOLDER_RAW}/minutes_info.xlsx")
df_info["DataReferencia"] = pd.to_datetime(df_info["DataReferencia"]).dt.date

In [122]:
df_info["nroReuniao"] = ""

for index, row in df_info.iterrows():
    try:
        nroReuniao = row["Titulo"][0:3]
        nroReuniao = int(nroReuniao)
    except:
        nroReuniao = row["Titulo"][0:2]
        nroReuniao = int(nroReuniao)

    df_info.at[index, "nroReuniao"] = nroReuniao

In [123]:
df_info["DataReferencia"] = df_info["nroReuniao"].map(df_minutes_date["dataReferencia"])
df_info["DataPublicacao"] = df_info["nroReuniao"].map(df_minutes_date["dataPublicacao"])	

In [ ]:
df_info.as_excel(f"{FOLDER_RAW}/minutes_info.xlsx", index=False)